In [1]:
%pip install yfinance pandas pandas_ta matplotlib reportlab fpdf

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
from datetime import datetime
from pathlib import Path
import os

# === CONFIGURAR CACHE ===
CACHE_DIR = Path("cache_prices")
CACHE_DIR.mkdir(exist_ok=True)

def obter_dados_com_cache(ticker, period="2y", interval="1d"):
    """
    Baixa dados do yfinance com cache local.
    Reutiliza dados históricos e atualiza apenas dados recentes.
    Compara DATAS (não horários) para determinar se cache está atualizado.
    """
    cache_file = CACHE_DIR / f"{ticker}_{interval}.pkl"
    
    try:
        # Se cache existe, carregar dados
        if cache_file.exists():
            df_cache = pd.read_pickle(cache_file)
            ultima_data = df_cache.index[-1]
            
            # Comparar apenas DATAS (sem hora) para dados diários
            data_ultima = pd.Timestamp(ultima_data.date())
            data_hoje = pd.Timestamp(datetime.now().date())
            
            # Se o cache tem dado de hoje, usa cache
            if data_ultima >= data_hoje:
                print(f"  ✓ Cache atualizado (até {data_ultima.date()})")
                return df_cache
            
            # Baixar dados novos desde última data no cache
            print(f"  ↻ Atualizando cache (último: {data_ultima.date()}, hoje: {data_hoje.date()})...")
            df_novo = yf.download(ticker, start=ultima_data, period=period, interval=interval, progress=False)
            
            if not df_novo.empty:
                # Combinar removendo duplicatas
                df_novo = df_novo[~df_novo.index.isin(df_cache.index)]
                df = pd.concat([df_cache, df_novo]).sort_index()
                df.to_pickle(cache_file)
                return df
            return df_cache
        
        else:
            # Primeiro download - buscar período completo
            print(f"  ↓ Baixando {period}...")
            df = yf.download(ticker, period=period, interval=interval, progress=False)
            if not df.empty:
                df.to_pickle(cache_file)
            return df
            
    except Exception as e:
        print(f"  ⚠️  Erro ao gerenciar cache: {e}")
        # Fallback: apenas baixar sem cache
        return yf.download(ticker, period=period, interval=interval, progress=False)

# Parâmetros do MACD
FAST = 12
SLOW = 26
SIGNAL = 9

# Criar pasta para gráficos
os.makedirs("macd_graficos", exist_ok=True)

def calcular_macd(data):
    data['EMA_fast'] = data['Close'].ewm(span=FAST, adjust=False).mean()
    data['EMA_slow'] = data['Close'].ewm(span=SLOW, adjust=False).mean()
    data['MACD_12_26_9'] = data['EMA_fast'] - data['EMA_slow']
    data['MACDs_12_26_9'] = data['MACD_12_26_9'].ewm(span=SIGNAL, adjust=False).mean()
    return data

def detectar_ultimo_cruzamento(data):
    for i in range(len(data)-1, 0, -1):
        macd_antes = data['MACD_12_26_9'].iloc[i - 1]
        sinal_antes = data['MACDs_12_26_9'].iloc[i - 1]
        macd_agora = data['MACD_12_26_9'].iloc[i]
        sinal_agora = data['MACDs_12_26_9'].iloc[i]
        if macd_antes < sinal_antes and macd_agora > sinal_agora:
            return 'Altista', data.index[i], float(data['Close'].iloc[i].item())
        elif macd_antes > sinal_antes and macd_agora < sinal_agora:
            return 'Baixista', data.index[i], float(data['Close'].iloc[i].item())
    return None, None, None

def gerar_grafico(data, ativo, tipo, data_cruz, preco_cruz):
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data['MACD_12_26_9'], label='MACD', linewidth=1.5)
    plt.plot(data.index, data['MACDs_12_26_9'], label='Signal', linewidth=1.2)

    # Destaque visual no ponto de cruzamento
    if data_cruz in data.index:
        y = data.loc[data_cruz, 'MACD_12_26_9']
        cor = 'green' if tipo == 'Altista' else 'red'
        plt.scatter(data_cruz, y, color=cor, s=60, label=f'Cruzamento {tipo}', zorder=5)

    plt.title(f'MACD - {ativo}')
    plt.legend()
    plt.grid(True)
    caminho = f'macd_graficos/{ativo}.png'
    plt.savefig(caminho)
    plt.close()
    return caminho

def criar_pdf(altistas, baixistas):
    today = datetime.now().strftime('%Y-%m-%d')
    BASE_DIR = Path.home() / 'Library/CloudStorage/OneDrive-Pessoal/Python/Financas'
    output_dir = BASE_DIR / '00_Relatorios'
    output_dir.mkdir(parents=True, exist_ok=True)
    pdf_path = output_dir / f'MACD_diario_B3_{today}.pdf'
    
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 10, "Relatório de Cruzamentos MACD - Diário B3", ln=True, align="C")

    # Tabela Altistas
    if altistas:
        pdf.set_font("Helvetica", "B", 12)
        pdf.ln(10)
        pdf.cell(0, 10, "Cruzamentos Altistas", ln=True)
        pdf.cell(50, 10, "Ativo", border=1)
        pdf.cell(45, 10, "Data", border=1)
        pdf.cell(45, 10, "Fechamento", border=1, ln=True)

        pdf.set_font("Helvetica", "", 12)
        for ativo, data, preco in altistas:
            pdf.cell(50, 10, ativo, border=1)
            pdf.cell(45, 10, data.strftime('%Y-%m-%d'), border=1)
            pdf.cell(45, 10, f"R$ {preco:.2f}", border=1, ln=True)

    # Tabela Baixistas
    if baixistas:
        pdf.set_font("Helvetica", "B", 12)
        pdf.ln(10)
        pdf.cell(0, 10, "Cruzamentos Baixistas", ln=True)
        pdf.cell(50, 10, "Ativo", border=1)
        pdf.cell(45, 10, "Data", border=1)
        pdf.cell(45, 10, "Fechamento", border=1, ln=True)

        pdf.set_font("Helvetica", "", 12)
        for ativo, data, preco in baixistas:
            pdf.cell(50, 10, ativo, border=1)
            pdf.cell(45, 10, data.strftime('%Y-%m-%d'), border=1)
            pdf.cell(45, 10, f"R$ {preco:.2f}", border=1, ln=True)

    # Gráficos
    for ativo, data, preco in altistas + baixistas:
        tipo = 'Altista' if (ativo, data, preco) in altistas else 'Baixista'
        pdf.add_page()
        pdf.set_font("Helvetica", "B", 12)
        pdf.cell(0, 10, f'Gráfico MACD - {ativo}', ln=True)
        pdf.image(f"macd_graficos/{ativo}.png", x=10, y=30, w=190)

    pdf.output(str(pdf_path))

def main():
    with open("ativos_b3.txt", "r") as f:
        ativos = [linha.strip().upper() for linha in f if linha.strip()]

    print(f"📋 Ativos carregados: {len(ativos)}")
    print(f"🎯 Buscando cruzamentos do MACD...\n")

    altistas = []
    baixistas = []

    for ativo in ativos:
        try:
            df = obter_dados_com_cache(ativo, period="2y", interval="1d")
            if df.empty:
                print(f"  ⚠️  Sem dados para {ativo}. Pulando...")
                continue
            df = calcular_macd(df)
            tipo, data_cruz, preco_cruz = detectar_ultimo_cruzamento(df)

            # Filtro: apenas cruzamento no último fechamento
            if tipo and data_cruz == df.index[-1]:
                if tipo == "Altista":
                    altistas.append((ativo, data_cruz, preco_cruz))
                    print(f"  ✓ {ativo}: Cruzamento ALTISTA em {data_cruz.date()} @ {preco_cruz:.2f}")
                else:
                    baixistas.append((ativo, data_cruz, preco_cruz))
                    print(f"  ✗ {ativo}: Cruzamento BAIXISTA em {data_cruz.date()} @ {preco_cruz:.2f}")

                gerar_grafico(df, ativo, tipo, data_cruz, preco_cruz)
            else:
                print(f"  ~ {ativo}: Sem cruzamento (ou não no último fechamento)")
        except Exception as e:
            print(f"  ❌ {ativo}: Erro — {e}")

    print(f"\n{'='*60}")
    print(f"📊 RESUMO FINAL")
    print(f"{'='*60}")
    print(f"✅ Cruzamentos ALTISTAS: {len(altistas)}")
    for ativo, data, preco in altistas:
        print(f"   • {ativo:8s} em {data.date()} @ {preco:>10.2f}")

    print(f"\n❌ Cruzamentos BAIXISTAS: {len(baixistas)}")
    for ativo, data, preco in baixistas:
        print(f"   • {ativo:8s} em {data.date()} @ {preco:>10.2f}")

    if altistas or baixistas:
        criar_pdf(altistas, baixistas)
        today = datetime.now().strftime('%Y-%m-%d')
        print(f"\n✅ Relatório gerado com sucesso: MACD_diario_B3_{today}.pdf")
    else:
        print("\n⚠️  Nenhum cruzamento no último fechamento.")

if __name__ == "__main__":
    main()

Note: you may need to restart the kernel to use updated packages.
📋 Ativos carregados: 80
🎯 Buscando cruzamentos do MACD...

  ⚠️  Erro ao gerenciar cache: (<StringDtype(storage='python', na_value=nan)>, array(['Close', 'High', 'Low', 'Open', 'Volume'], dtype=object))
YF.download() has changed argument auto_adjust default to True



1 Failed download:
['ABEV3.SA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  ⚠️  Sem dados para ABEV3.SA. Pulando...
  ⚠️  Erro ao gerenciar cache: (<StringDtype(storage='python', na_value=nan)>, array(['Close', 'High', 'Low', 'Open', 'Volume'], dtype=object))



1 Failed download:
['ALOS3.SA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  ⚠️  Sem dados para ALOS3.SA. Pulando...
  ⚠️  Erro ao gerenciar cache: (<StringDtype(storage='python', na_value=nan)>, array(['Close', 'High', 'Low', 'Open', 'Volume'], dtype=object))


...

  ⚠️  Sem dados para ORCL34.SA. Pulando...
  ⚠️  Erro ao gerenciar cache: (<StringDtype(storage='python', na_value=nan)>, array(['Close', 'High', 'Low', 'Open', 'Volume'], dtype=object))



1 Failed download:
['P2LT34.SA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  ⚠️  Sem dados para P2LT34.SA. Pulando...
  ⚠️  Erro ao gerenciar cache: [Errno 60] Operation timed out



1 Failed download:
['TSMC34.SA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  ⚠️  Sem dados para TSMC34.SA. Pulando...

📊 RESUMO FINAL
✅ Cruzamentos ALTISTAS: 0

❌ Cruzamentos BAIXISTAS: 0

⚠️  Nenhum cruzamento no último fechamento.
